In [3]:
with open('constitucion_colombiana.txt', 'r') as f:
    text = f.readlines()

    f.close()
    
for line in text:
    if line == '\n':
        text.remove(line)

In [4]:
def text_remove_nl(text):
    for line in text:
        limit = len(line) - 1
        n_line = line[:limit]
        line = n_line

    return text
clean_text = text_remove_nl(text)
# Drop the title "Constitucion Politica de Colombia"
clean_text = clean_text[1:len(text)]
clean_text

['TITULO I\n',
 'DE LOS PRINCIPIOS FUNDAMENTALES\n',
 'Artículo 1. Colombia es un Estado social de derecho, organizado en forma de República unitaria, descentralizada, con autonomía de sus entidades territoriales, democrática, participativa y pluralista, fundada en el respeto de la dignidad humana, en el trabajo y la solidaridad de las personas que la integran y en la prevalencia del interés general.\n',
 'Artículo 2. Son fines esenciales del Estado: servir a la comunidad, promover la prosperidad general y garantizar la efectividad de los principios, derechos y deberes consagrados en la Constitución; facilitar la participación de todos en las decisiones que los afectan y en la vida económica, política, administrativa y cultural de la Nación; defender la independencia nacional, mantener la integridad territorial y asegurar la convivencia pacifica y la vigencia de un orden justo.\n',
 'Las autoridades de la República están instituidas para proteger a todas las personas residentes en Colo

In [10]:
def label_dict(name_importance):
    if name_importance == True:
        label_dict = {'name' : None, 'content': None}
    else:
        label_dict = {'content': None}

    return label_dict

def join_labels(text, label, name_importance=True):
    labeled_text = {}
    
    # Intialized the variable
    name_next = False
    
    for l in range(len(text)):
        line = text[l]
        # In all cases, the 0 is an start (also in chapters)
        if l == 0:
            # In case of articles, it does not add the name part
            labeled_text[line] = label_dict(name_importance)
            # Where it remembers the last label used
            last_label = line
            #In other case, this remains as False
            if name_importance == True: name_next = True
                
        else:  # For the next cases
            # Recognize for a new label
            if line[:len(label)].upper() == label.upper():
                # Creates the label on the dictionary
                labeled_text[line] = label_dict(name_importance)
                
                # Remembers the new label
                last_label = line
                if name_importance == True: name_next = True
                    
            else:
                if name_importance == True and name_next == True:
                    # In case that the name matters, like tittles and chapters
                    labeled_text[last_label]['name'] = line
                    
                    name_next = False
                else:
                    if labeled_text[last_label]['content'] == None:
                        labeled_text[last_label]['content'] = [line]
                    else:
                        labeled_text[last_label]['content'].append(line)

    return labeled_text

chapters = join_labels(clean_text, label = 'TITULO')
# for key in chapters:
#     print(key, chapters[key]['name'], chapters[key]['content'][:2])

print('\n------------------\n')
test_key = 'TITULO II\n'
test_text = chapters[test_key]['content']
print(test_text[-2])
print('\n------------------\n')

title_2 = chapters = join_labels(test_text, label = 'CAPITULO', name_importance = True)
for key in title_2:
    print(key,title_2[key]['content'], end='\n\n')


------------------

8. Proteger los recursos culturales y naturales del país y velar por la conservación de un ambiente sano,


------------------

CAPITULO 1
 ['Artículo 11. El derecho a la vida es inviolable. No habrá pena de muerte.\n', 'Artículo 12. Nadie será sometido a desaparición forzada, a torturas ni a tratos o penas crueles, inhumanos o degradantes.\n', 'Artículo 13. Todas las personas nacen libres e iguales ante la ley, recibirán la misma protección y trato de las autoridades y gozarán de los mismos derechos, libertades y oportunidades sin ninguna discriminación por razones de sexo, raza, origen nacional o familiar, lengua, religión, opinión política o filosófica.\n', 'El Estado promoverá las condiciones para que la igualdad sea real y efectiva y adoptara medidas en favor de grupos discriminados o marginados.\n', 'El Estado protegerá especialmente a aquellas personas que por su condición económica, física o mental, se encuentren en circunstancia de debilidad manifiesta y s